In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
label2id = {'negative':0,'positive':1}

train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train = train.replace({"label": label2id})
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test = test.replace({"label": label2id})
test.fillna('', inplace=True)

/tmp/ipykernel_7030/2093326092.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train = train.replace({"label": label2id})
/tmp/ipykernel_7030/2093326092.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace({"label": label2id})


In [4]:
train

,label,text,diff
0,0,Fix #2197 extended regex pattern example,diff --git a/docs/content/manual/manual.yml b/...
1,0,some test spec fixes,diff --git a/asset-pipeline-grails/src/test/gr...
2,1,"FILEUPLOAD-279: Introduce a system property, w...",diff --git a/.gitignore b/.gitignore\nnew file...
3,0,fix(documentation): remove route redeclaration...,diff --git a/doc/API/centreon-api-v2.yaml b/do...
4,0,[JENKINS-49744] - Generalize the permission ha...,diff --git a/src/main/java/com/synopsys/arc/je...
...,...,...,...
96,0,[maven-release-plugin] prepare release checkst...,diff --git a/pom.xml b/pom.xml\nindex 2043823....
97,0,Merge pull request #360 from joaosantos81/mast...,diff --git a/control/Session.php b/control/Ses...
98,0,Repairing: implement invalid node checksum fix.,diff --git a/libexfat/exfat.h b/libexfat/exfat...
99,1,[MJ2] Avoid index out of bounds access to pi->...,diff --git a/src/lib/openmj2/pi.c b/src/lib/op...


In [5]:
len(train)

101

In [6]:
train['label'].value_counts()

label
0    63
1    38
Name: count, dtype: int64

In [7]:
test['label'].value_counts()

label
0    6284
1    3727
Name: count, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
# model_id = "../../sentence-transformers/paraphrase-mpnet-base-v2"
model = SentenceTransformer('../../sentence-transformers/paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,0,Merge pull request #1015 from omniauth/make-su...,diff --git a/Gemfile b/Gemfile\nindex b0581c01...,"[-0.13004021, 0.1200651, -0.08801293, 0.104843..."
1,0,"Cleaned up rdpei channel, updated to current s...",diff --git a/channels/rdpei/client/rdpei_main....,"[-0.042629246, -0.061168876, 0.026884645, -0.0..."
2,0,Make remaining AST classes public. Fixes jkuhn...,diff --git a/src/main/java/ognl/ASTAdd.java b/...,"[0.045945346, 0.17283344, -0.029472524, 0.0503..."
3,0,Test custom field uploads on ticket create for...,diff --git a/t/rest2/tickets.t b/t/rest2/ticke...,"[-0.09309562, 0.22370622, -0.043404736, 0.1327..."
4,0,Merge branch '2.7' into master,diff --git a/.travis.yml b/.travis.yml\nindex ...,"[-0.1171597, -0.009933171, 0.051275834, -0.041..."
...,...,...,...,...
10006,1,Also unfold and split lazily\n\nThis has littl...,diff --git a/lib/mail/field.rb b/lib/mail/fiel...,"[-0.068634324, -0.053742584, -0.08566991, 0.12..."
10007,0,Adding a property 'jdk.xml.resolveExternalEnti...,diff --git a/src/org/apache/xerces/parsers/Sec...,"[-0.037466437, 0.21927911, -0.028538529, 0.022..."
10008,1,name might not be displayed correctly\n\nChang...,diff --git a/tine20/Addressbook/js/ContactGrid...,"[-0.04909946, 0.19766133, -0.055661, 0.1250326..."
10009,0,Trailing whitespace...\n\nMake Travis happy,diff --git a/airsonic-main/src/main/java/org/a...,"[-0.020277845, 0.023705391, -0.14176506, 0.149..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [13]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Precision: 0.780282647816306
Logistic Regression Recall: 0.7833383278393767
Logistic Regression Recall: 0.7800807003527102
Logistic Regression Accuracy: 0.7833383278393767
